In [5]:
import cv2
import numpy as np

# for this object, we need to provide 6 inputs for the filter to run.
# They are: the acceleration along x and y , uncertainty in acceleration, the uncertainty of position measurement along x and y, and a variable for the change in time.
    
class KF(object):
    def __init__(self,dt,acc_x,acc_y,sigma_x,sigma_y,sigma_acc):
        
        self.dt = dt
        self.acc = np.array([[acc_x],[acc_y]])

        # initialize the first state
        self.X = np.zeros([4,1])
        #initialize the covariance matrix by guessing a value. The guess can improve the accuracy of the initial predicted state
        self.P = np.eye(4)
        
        #Now we need to define the matrics F, B, and H in order to have everything we need to the 2 Kalman filter equations
        self.B = np.array([[0.5*(self.dt**2), 0],[0.5*(self.dt**2),0],[self.dt,0],[0,self.dt]])
        self.F = np.array([[1, 0, self.dt, 0],[0, 1, 0, self.dt],[0, 0, 1, 0],[0, 0, 0, 1]])
        self.H = np.array([[1, 0, 0, 0],[0, 1, 0, 0]])

        #Later when we try to update the state, we will need the covariance matrics for the noises in the measurements and process
        self.R = np.array([[sigma_x**2,0],[0,sigma_y**2]])
        self.Q = np.array([[0.25*(self.dt**4), 0, 0.5*(self.dt**3), 0],[0,0.25*(self.dt**4), 0,0.5*(self.dt**3)],[0.5*(self.dt**3), 0, self.dt**2, 0],[0, 0.5*(self.dt**3), 0, self.dt**2]])*sigma_acc**2

    def predict(self):
        self.X = np.dot(self.F,self.X)+np.dot(self.B,self.acc)
        self.P = np.dot(np.dot(self.F,self.P),self.F.T)+self.Q
        return self.X

    def update(self, z):
        S = np.dot(np.dot(self.H,self.P),self.H.T)+self.R
        K = np.dot(np.dot(self.P,self.H.T),np.linalg.inv(S))
        self.P = np.dot((np.eye(4)-np.dot(K,self.H)),self.P)
        self.X = self.X + np.dot(K, (z - np.dot(self.H, self.X)))
        return self.X


VideoCap = cv2.VideoCapture('dot.mp4')


KalmanFilter = KF(0.5, 2, 3, 0.2, 0.3,0.5)
#performance improves as dt increases
while(True):
    _, frame = VideoCap.read()
    blurred = cv2.GaussianBlur(frame,(5,5),0)
    canny = cv2.Canny(blurred,  50, 190, 3)
    _, img_thresh = cv2.threshold(canny, 254, 255,cv2.THRESH_BINARY)
    _,contours,Hierarchy = cv2.findContours(img_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    centroids=[]
    for i in contours:
        (x, y), _ = cv2.minEnclosingCircle(i)
        centroids.append(np.array([[int(x)], [int(y)]]))
    if (len(centroids) > 0):
        cv2.circle(frame, (int(centroids[0][0]), int(centroids[0][1])), 10, (0, 255, 0), 2)
        State_predicted = KalmanFilter.predict()
        (m,n) = State_predicted[0:2] # we only need the position values which are the first 2 elements in the X_k array
        cv2.circle(frame, (m+3,n+3), 10, (0, 0, 255), 2)
        print(centroids[0])
        State_updated = KalmanFilter.update(centroids[0])
        (a,b) = State_updated[0:2]
        cv2.circle(frame, (a+6,b+6), 10, (255, 0, 0), 2)
        cv2.putText(frame, "position measured", (centroids[0][0], centroids[0][1]), 0, 0.5, (0,255,0), 2)
        cv2.putText(frame, "position predicted", (m+3, n+3), 0, 0.5, (0,0,255), 2)
        cv2.putText(frame, "position updated", (a+6,b+6), 0, 0.5, (255,0,0), 2)
    cv2.imshow('image', frame)

    if cv2.waitKey(1) & 0xFF == ord('x'):
        VideoCap.release()
        cv2.destroyAllWindows()
        break



[[1055]
 [ 455]]
[[1075]
 [ 431]]
[[1095]
 [ 407]]
[[1114]
 [ 384]]
[[1134]
 [ 360]]
[[1154]
 [ 336]]
[[1180]
 [ 310]]
[[1201]
 [ 282]]
[[1222]
 [ 259]]
[[1228]
 [ 250]]
[[1251]
 [ 227]]
[[1264]
 [ 209]]
[[1276]
 [ 125]]
[[1264]
 [ 120]]
[[1245]
 [ 132]]
[[1222]
 [ 127]]
[[1237]
 [ 140]]
[[1191]
 [ 132]]
[[1168]
 [ 134]]
[[1155]
 [ 136]]
[[1143]
 [ 136]]
[[1118]
 [ 142]]
[[1100]
 [ 145]]
[[1082]
 [ 148]]
[[1063]
 [ 151]]
[[1039]
 [ 161]]
[[1027]
 [ 156]]
[[1011]
 [ 159]]
[[991]
 [162]]
[[972]
 [164]]
[[954]
 [167]]
[[936]
 [170]]
[[918]
 [173]]
[[894]
 [176]]
[[882]
 [179]]
[[857]
 [188]]
[[858]
 [187]]
[[868]
 [197]]
[[881]
 [212]]
[[887]
 [216]]
[[896]
 [225]]
[[910]
 [237]]
[[913]
 [251]]
[[925]
 [255]]
[[932]
 [257]]
[[943]
 [273]]
[[955]
 [289]]
[[961]
 [293]]
[[974]
 [309]]
[[982]
 [312]]
[[989]
 [326]]
[[999]
 [340]]
[[1012]
 [ 347]]
[[1016]
 [ 356]]
[[1030]
 [ 359]]
[[1040]
 [ 368]]
[[1046]
 [ 371]]
[[1057]
 [ 387]]
[[1066]
 [ 397]]
[[1039]
 [ 399]]
[[1011]
 [ 411]]
[[981]
 [40

 [ 169]]
[[1123]
 [ 168]]
[[1123]
 [ 168]]
[[1126]
 [ 174]]
[[1124]
 [ 167]]
[[1123]
 [ 167]]
[[1124]
 [ 167]]
[[1124]
 [ 172]]
[[1123]
 [ 166]]
[[1132]
 [ 163]]
[[1124]
 [ 164]]
[[1132]
 [ 162]]
[[1124]
 [ 163]]
[[1127]
 [ 159]]
[[1132]
 [ 160]]
[[1125]
 [ 168]]
[[1125]
 [ 161]]
[[1134]
 [ 162]]
[[1126]
 [ 162]]
[[1127]
 [ 163]]
[[1120]
 [ 166]]
[[1120]
 [ 164]]
[[1129]
 [ 152]]
[[1126]
 [ 158]]
[[1130]
 [ 161]]
[[1120]
 [ 162]]
[[1133]
 [ 154]]
[[1133]
 [ 154]]
[[1133]
 [ 154]]
[[1134]
 [ 153]]
[[1127]
 [ 154]]
[[1134]
 [ 152]]
[[1134]
 [ 152]]
[[1127]
 [ 153]]
[[1127]
 [ 152]]
[[1127]
 [ 153]]
[[1127]
 [ 152]]
[[1128]
 [ 151]]
[[1127]
 [ 151]]
[[1121]
 [ 153]]
[[1128]
 [ 150]]
[[1127]
 [ 149]]
[[1128]
 [ 154]]
[[1127]
 [ 149]]
[[1130]
 [ 142]]
[[1128]
 [ 156]]
[[1123]
 [ 148]]
[[1123]
 [ 147]]
[[1128]
 [ 146]]
[[1128]
 [ 146]]
[[1128]
 [ 145]]
[[1123]
 [ 145]]
[[1128]
 [ 144]]
[[1128]
 [ 144]]
[[1128]
 [ 143]]
[[1128]
 [ 143]]
[[1136]
 [ 216]]
[[1150]
 [ 300]]
[[1165]
 [ 375]]
[[117